In [1]:
#Setup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob, pickle, joblib, os, uproot
from tqdm import tqdm_notebook

import rawio.IO as io
from scdmsPyTools.Traces.Filtering import *
from scdmsPyTools.Traces.Noise import *
from scdmsPyTools.TES.Templates import *
from scdmsPyTools.Traces.Stats import *
#from scdmsPyTools.BatTools.IO import *
rawdir = "/data/midas/"
procdir = "/data/midas/proc"

In [2]:
#Select a file.
series='48893'
csv = [x[0] for x in pd.read_csv('~/git/R76Firmware/DCRC_Compatibility/manual_48893.txt',dtype=str,comment='#').values.tolist()]
#txt = [x[0] for x in pd.read_csv('~/git/R76Firmware/DCRC_Compatibility/towertest.txt',dtype=str).values.tolist()]

#More setup, dependent on series number.
procseries=series
filepath = rawdir+series

events = io.getRawEvents(rawdir,series)
try:
    stamps = [f'{QI[0]:04X}' for QI in events['Z6']['QI'][events['Z6']['PA'].keys()[0][0]]]
except KeyError:
    print(events.keys(),len(events))

del events

The file is: /data/midas//48893/RUN48893_F0000.mid.gz
RawDataReader::ParseSeriesAndDumpFromRawPath: WARNING could not parse filename for series and dump, not getting them from filename, OK if the info is in the data stream

Opened raw midas file: /data/midas//48893/RUN48893_F0000.mid.gz
RawDataReader: Extracted series from ODB: 72406211708


RawDataReader::ERROR Couldn't extract dump number from ODB settings. Setting to 0.


print(csv['timestamp'],[f'{i:x}' for i in stamps])
print(type(csv['timestamp'][0]),type(f'{100:x}'))
print('1337' in csv['timestamp'].values)

In [3]:
print(stamps,"\n====\n",csv,len(stamps),len(csv))

['AED9', '527E', '68CE', '9EF4', 'EEF8', '4CCA', '69FC', '0A3A', 'C0F2', '0AC8', '3DF7', '9B3A', '4BAE', '4018', 'CE5F', '4DD6', 'A397', 'C452', 'AACF', '084D', 'D7EE', '0C60', '478F', '4DD3', '8AD0', '452D', '58A5', 'AA9B', 'D7A6', 'EC66', '436E', 'CC61', 'FEB1', 'FEED', 'B945'] 
====
 ['527E', '68CF', '9EF4', 'EEF8', '04BD', '4CCB', '54F3', '69FD', '0A3A', 'A2A3', 'C0F2', '0AC9', '3DF7', '9B3A', '4BAF', 'B960', '4019', 'CE5F', '4DD7', 'A99F', 'A398', 'A1BC', '00CC', 'C452', 'AAD0', '084E', 'D7EF', '0C61', '478F', '4DD3', '89B4', 'F0DB', '8AD0', '452E', '58A5', 'AA9B', 'D7A7', '2726', 'EC66', 'FA10', '0322', '436F', 'A970', '902A', '2307', 'CC61', 'FEB2', 'FEED', 'B946'] 35 49


In [4]:
#for x in range(385): if hex(x)[2:] not in stamps: print(hex(x)[2:])
print(np.diff([int(stamp,16) for stamp in stamps]))

[-23643   5712  13862  20484 -41518   7474 -24514  46776 -46634  13103
  23875 -20364  -2966  36423 -32905  21953   8379  -6531 -41602  53153
 -52110  15151   1604  15613 -17827   4984  20982  11531   5312 -43256
  35059  12880     60 -17832]


In [5]:
print(hex(len(stamps)),len(stamps)/385)

0x23 0.09090909090909091


In [6]:
shared = [x for x in csv if x in stamps]
print(shared)
csvshared = csv[csv.index(shared[0]):]#csv.index(shared[-1])+1]
stampshared = stamps[stamps.index(shared[0]):]#stamps.index(shared[-1])+1]
#stampshared = [x for x in stampshared if x != '014A']

['527E', '9EF4', 'EEF8', '0A3A', 'C0F2', '3DF7', '9B3A', 'CE5F', 'C452', '478F', '4DD3', '8AD0', '58A5', 'AA9B', 'EC66', 'CC61', 'FEED']


In [7]:
print(csvshared,stampshared)

['527E', '68CF', '9EF4', 'EEF8', '04BD', '4CCB', '54F3', '69FD', '0A3A', 'A2A3', 'C0F2', '0AC9', '3DF7', '9B3A', '4BAF', 'B960', '4019', 'CE5F', '4DD7', 'A99F', 'A398', 'A1BC', '00CC', 'C452', 'AAD0', '084E', 'D7EF', '0C61', '478F', '4DD3', '89B4', 'F0DB', '8AD0', '452E', '58A5', 'AA9B', 'D7A7', '2726', 'EC66', 'FA10', '0322', '436F', 'A970', '902A', '2307', 'CC61', 'FEB2', 'FEED', 'B946'] ['527E', '68CE', '9EF4', 'EEF8', '4CCA', '69FC', '0A3A', 'C0F2', '0AC8', '3DF7', '9B3A', '4BAE', '4018', 'CE5F', '4DD6', 'A397', 'C452', 'AACF', '084D', 'D7EE', '0C60', '478F', '4DD3', '8AD0', '452D', '58A5', 'AA9B', 'D7A6', 'EC66', '436E', 'CC61', 'FEB1', 'FEED', 'B945']


In [8]:
for x in list(zip(csvshared,stampshared)): 
    if(x[0][:2] == x[1][:2]): print(x,"*")
    else: print(x)
print(shared)

('527E', '527E') *
('68CF', '68CE') *
('9EF4', '9EF4') *
('EEF8', 'EEF8') *
('04BD', '4CCA')
('4CCB', '69FC')
('54F3', '0A3A')
('69FD', 'C0F2')
('0A3A', '0AC8') *
('A2A3', '3DF7')
('C0F2', '9B3A')
('0AC9', '4BAE')
('3DF7', '4018')
('9B3A', 'CE5F')
('4BAF', '4DD6')
('B960', 'A397')
('4019', 'C452')
('CE5F', 'AACF')
('4DD7', '084D')
('A99F', 'D7EE')
('A398', '0C60')
('A1BC', '478F')
('00CC', '4DD3')
('C452', '8AD0')
('AAD0', '452D')
('084E', '58A5')
('D7EF', 'AA9B')
('0C61', 'D7A6')
('478F', 'EC66')
('4DD3', '436E')
('89B4', 'CC61')
('F0DB', 'FEB1')
('8AD0', 'FEED')
('452E', 'B945')
['527E', '9EF4', 'EEF8', '0A3A', 'C0F2', '3DF7', '9B3A', 'CE5F', 'C452', '478F', '4DD3', '8AD0', '58A5', 'AA9B', 'EC66', 'CC61', 'FEED']


In [9]:
print(len(stamps)-len(set(stamps)))
print(stamps)

0
['AED9', '527E', '68CE', '9EF4', 'EEF8', '4CCA', '69FC', '0A3A', 'C0F2', '0AC8', '3DF7', '9B3A', '4BAE', '4018', 'CE5F', '4DD6', 'A397', 'C452', 'AACF', '084D', 'D7EE', '0C60', '478F', '4DD3', '8AD0', '452D', '58A5', 'AA9B', 'D7A6', 'EC66', '436E', 'CC61', 'FEB1', 'FEED', 'B945']


In [10]:
repeats = [x for x in stamps if stamps.count(x) > 1]
print(set(repeats), (len(repeats)-len(set(repeats)))/len(set(repeats)))

for x in stamps:
    print(x+(stamps.count(x)-1)*"*")

ZeroDivisionError: division by zero

So it seems like none of these are aligned. I'm switching to using a FIFO instead of RAM so that I can store a handful of values -- maybe it's getting overwritten by one of the next few before it has a chance to write to the packet, so a different value is ultimately saved.
- On next iteration, just set the FIFO to max size -- we actually need more space than we have now, since the rate could go much higher, and if it's too small then the whole system will fail. There's a limit to this, where if we can handle that trigger rate, we can handle anything above it (because of the inhibit function), but there's not really any reason to try to keep this small, is there?

In [ ]:
for i,x in enumerate(stamps[stamps.index(shared[0]):]):
    print("{0:016b}".format(int(x,16) ^ int(csv[csv.index(shared[0]):][i],16)))

In [ ]:
for i,x in enumerate(stamps[stamps.index(shared[0]):]):
    print(x == csv[csv.index(shared[0]):][i])

Upside: fewer obvious red flag values like beef or 1337. I think the repetition is a red flag. Is there anything in the code that tells the DCRC to retrieve the same value again, maybe?

In [ ]:
print(kill_dupes)

# Algorithm
from R76Analysis/example_scripts/time_difference...

In [ ]:
from itertools import zip_longest

bold_text = "\033[1m"
reset_format = "\033[0m"
title = "Merge and look for doubles:"
print(f"{bold_text}{title}{reset_format}")
print("pros and cons: this method is very simple and sorts well, but it deletes a lot of good data too")

merged_list = [item for pair in zip_longest(stamps,csv, fillvalue=None) for item in pair if item is not None]

for i,s in enumerate(merged_list):
    print(i,s)

trash=set()
i = 0
while i + 2 < len(merged_list):
    if merged_list[i] == merged_list[i + 1]:
        #print(":)")
        i += 2
    else:
        trash.add(i)
        trash.add(i + 1)
        trash.add(i + 2)
        i += 3
            
for i in sorted(trash, reverse=True):
    del merged_list[i]

print()
print("New:")
for i,s in enumerate(merged_list):
    print(i,s)

todo:
- let it search the rest of the list, in case there are extras to be dropped
- tolerance +/- 1?